In [1]:
import tensorflow as tf
from gensim.models import Word2Vec
import pandas as pd
from textblob import TextBlob
import numpy as np

c:\users\princ3\anaconda3\envs\tfconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Importing the word2vec models

In [2]:
word_vec = Word2Vec.load('model.bin')    # word embedding
# char_vec = Word2Vec.load('model_c.bin')  # charector embedding

### Importing the dataset

In [3]:
df_contxt = pd.read_pickle('./input/squad_contxt.pkl') # context
df_qas = pd.read_pickle('./input/squad_qas.pkl')       # question and answers

### Preprocessing data

In [4]:
class prep_data():
    """
    
    """
    def __init__(self,contxt = df_contxt,qas = df_qas, word_e = word_vec):
        
        self.contxt = contxt
        self.qas = qas
        self.char_e = char_e
        self.word_e = word_e

    def C2V(self,passage):   # this function converts context 2 vector with <EOS> tag
        tb_p = TextBlob(passage)
        p2v = []
        for i in tb_p.sentences:
            words = TextBlob(str(i)+" E-O-S").words
            for j in words:
                p2v.append(word_vec[j])
        return np.array(p2v)
    
    def QA2V(self,question_no):  # this function converts question 2 vector
        tb_a = TextBlob(self)
        
        

###  Building the model

In [24]:
class gru_models():
    """
        It's a modified version of normal GRU
    """
    def __init__(self,time_steps,init_state,input_size,tag,
                 output_size,question=None,final_mem=None,input_seq=None):
        
        # inputs required for sequential memory vectors e_i 
        self.time_steps = time_steps              # equal to max no concepts
        self.init_state = init_state              # initial secondary input ideally zero tensor
        self.input_seq = input_seq                # input seq to be evaluated
        self.input_size = input_size              # size of input vector 
        self.output_size = output_size            # size of output required
        
        # inputs required for Answer modules
        self.question = question
        self.final_mem = final_mem
        self.word_weight = tf.get_variable("word_w_{}".format(tag),[output_size,output_size],dtype=tf.float64)
        self.word_bias = tf.get_variable("word_b_{}".format(tag),[output_size,1],dtype=tf.float64)
        
        # parameters of gru
        self.W_z,self.W_r,self.W_h = tf.get_variable("update_w_{}".format(tag),[output_size,input_size],dtype=tf.float64),tf.get_variable("reset_w_{}".format(tag),[output_size,input_size],dtype=tf.float64),tf.get_variable("out_w_{}".format(tag),[output_size,input_size],dtype=tf.float64)
        self.U_z,self.U_r,self.U_h = tf.get_variable("update_u_{}".format(tag),[output_size,output_size],dtype=tf.float64),tf.get_variable("reset_u_{}".format(tag),[output_size,output_size],dtype=tf.float64),tf.get_variable("out_u_{}".format(tag),[output_size,output_size],dtype=tf.float64)
        self.b_z,self.b_r,self.b_h = tf.get_variable("update_b_{}".format(tag),[output_size,1],dtype=tf.float64),tf.get_variable("reset_b_{}".format(tag),[output_size,1],dtype=tf.float64),tf.get_variable("out_b_{}".format(tag),[output_size,1],dtype=tf.float64)
    
    """ Normal GRU unit """
    def gru_unit(self,h_prev,steps,out=[]):
        index = steps - self.time_steps 
        input_vector = self.input_seq[index]
        # operations 
        z_t = tf.nn.sigmoid(tf.matmul(self.W_z,input_vector)+tf.matmul(self.U_z,h_prev)+self.b_z)
        r_t = tf.nn.sigmoid(tf.matmul(self.W_r,input_vector)+tf.matmul(self.U_r,h_prev)+self.b_r)
        h_t = tf.multiply(z_t,h_prev)+tf.multiply(1.-z_t,tf.nn.sigmoid(tf.matmul(self.W_h,input_vector)+tf.matmul(self.U_h,tf.multiply(r_t,h_prev))+self.b_h))
#         h_t = (self.scalar_values[index]*h_t) + ((1-self.scalar_values[index])*h_prev)
        out.append(h_t)
        self.time_steps -= 1
        if self.time_steps == 0:
            return(out)
        else:
            return self.gru_unit(h_prev = h_t,out = out,steps = steps) 
    
    
    """
    it's a modified GRU !!!
    """
    def mod_gru_unit(self,h_prev,steps,scalar_values,out=[],index=0):
#         print(index)
        input_vector = self.input_seq[index]
        # operations
        z_t = tf.nn.sigmoid(tf.matmul(self.W_z,input_vector)+tf.matmul(self.U_z,h_prev)+self.b_z)
        r_t = tf.nn.sigmoid(tf.matmul(self.W_r,input_vector)+tf.matmul(self.U_r,h_prev)+self.b_r)
        h_t = tf.multiply(z_t,h_prev)+tf.multiply(1.-z_t,tf.nn.sigmoid(tf.matmul(self.W_h,input_vector)+tf.matmul(self.U_h,tf.multiply(r_t,h_prev))+self.b_h))
        h_t = (scalar_values[index]*h_t) + ((1-scalar_values[index])*h_prev)
        out.append(h_t)
        index += 1
        if steps-index == 0:
            return(out)
        else:
            return self.mod_gru_unit(h_prev = h_t,out = out,steps = steps,
                                     scalar_values=scalar_values,index=index) 
        
    """ It's a hybrid GRU for answer module """
    def answer_gru_unit(self,h_prev,word_prev,steps,out=[]):
        index = steps - self.time_steps 
        print(index)
#         print(np.shape(word_prev),np.shape(self.question))
        input_vector = tf.concat([word_prev,self.question],axis=0)
        # operations
        z_t = tf.nn.sigmoid(tf.matmul(self.W_z,input_vector)+tf.matmul(self.U_z,h_prev)+self.b_z)
        r_t = tf.nn.sigmoid(tf.matmul(self.W_r,input_vector)+tf.matmul(self.U_r,h_prev)+self.b_r)
        h_t = tf.multiply(z_t,h_prev)+tf.multiply(1.-z_t,tf.nn.sigmoid(tf.matmul(self.W_h,input_vector)+tf.matmul(self.U_h,tf.multiply(r_t,h_prev))+self.b_h))
        word_pred = tf.nn.softmax(tf.matmul(self.word_weight,h_t)+self.word_bias)
#         print(h_t)
#         print(word_pred)
        out.append(word_pred)
        self.time_steps -= 1
        if self.time_steps == 0:
            return(out)
        else:
            return self.answer_gru_unit(h_prev = h_t,word_prev = word_pred,out = out,steps = steps) 

In [34]:
class DMN_QA():
    """
    the word vectors have size 100
    the max no of words in a sentence is 120
    """
    
    def __init__(self):
        self.MAX_P_WORDS = 120       # max no of words in a passage
        self.WORD_VEC_LEN = 100      # word vector length
        self.MAX_Q_WORDS = 10        # max no of words in question
        self.MAX_NO_CONCEPTS = 5      # max no of concepts
        
        # variables and placeholders
        self.Passage = tf.placeholder(shape=(self.MAX_P_WORDS,self.WORD_VEC_LEN,1),dtype = tf.float64)
        self.EOS_Tag = tf.placeholder(tf.float64)
        self.Question = tf.placeholder(shape=(self.MAX_Q_WORDS,self.WORD_VEC_LEN,1),dtype = tf.float64) #sequence for question
        self.Answer = tf.placeholder(tf.float64)
        
        # weigts for scalar kernel and memory vector
        self.W_b = tf.get_variable("W_b",shape=(100,100),dtype=tf.float64)
        self.W_1 = tf.get_variable("W_1",shape=(200,702),dtype=tf.float64)
        self.b_1 = tf.get_variable("b_1",shape=(200,1),dtype=tf.float64)
        self.W_2 = tf.get_variable("W_2",shape=(1,200),dtype=tf.float64)
        self.b_2 = tf.get_variable("b_2",shape=(1,1),dtype=tf.float64)
        
        # initial memory state
        self.memory = tf.placeholder(shape=(100,1),dtype=tf.float64)
    
    def build(self):
        
        # question module 
        Q_module = gru_models(time_steps=self.MAX_Q_WORDS,init_state=np.zeros([self.WORD_VEC_LEN,1]),
                              input_size=100,output_size=100,input_seq=self.Question,tag=1)
        Q_out = Q_module.gru_unit(h_prev=np.zeros([self.WORD_VEC_LEN,1]),steps=self.MAX_Q_WORDS,out=[])
        
        self.question = Q_out[-1] # final representation of question
        #passage module
        P_module = gru_models(time_steps=self.MAX_P_WORDS,init_state=np.zeros([self.WORD_VEC_LEN,1]),
                              input_size=100,output_size=100,input_seq=self.Passage,tag=2)
        P_out = P_module.gru_unit(h_prev=np.zeros([self.WORD_VEC_LEN,1]),steps=self.MAX_P_WORDS,out=[])
#         print(len(P_out))
        #episodic memory module
        concepts = P_out[:3]  # selected E-O-S tags from P_out
        epi_mod = gru_models(time_steps=2,init_state=np.zeros([self.WORD_VEC_LEN,1]),
                              input_size=100,output_size=100,input_seq=self.Passage,tag=3)
        y = self.epi_mem_mod(concepts,len(concepts),epi_mod,[])
        print(y)
        return y
    
    def epi_mem_mod(self,concepts,no_of_iterations,mode,mem_states=[]):
        scalar_vector = []
        for i in concepts:
            scalar_vector.append(self.scalar_gate_value(i))
        
        concepts_out = mode.mod_gru_unit(h_prev = self.memory,steps=len(concepts),scalar_values=scalar_vector,
                          out=[])
        self.memory = concepts_out[-1]
        mem_states.append(concepts_out[-1])
#         print(no_of_iterations)
        if no_of_iterations == 0:
            print(mem_states)
            return mem_states
        else:
            self.epi_mem_mod(concepts,no_of_iterations-1,
                        mode,mem_states)
            
    def scalar_gate_value(self,concept):  # debugged and perfect
        z_vector = tf.concat((concept,self.memory),axis=0)
        z_vector = tf.concat((z_vector,self.question),axis=0)
        z_vector = tf.concat((z_vector,tf.multiply(concept,self.question)),axis=0)
        z_vector = tf.concat((z_vector,tf.multiply(concept,self.memory)),axis=0)
        z_vector = tf.concat((z_vector,concept-self.question),axis=0)
        z_vector = tf.concat((z_vector,concept-self.memory),axis=0)
        z_vector = tf.concat((z_vector,tf.matmul(tf.transpose(concept),tf.matmul(self.W_b,self.question))),axis=0)
        z_vector = tf.concat((z_vector,tf.matmul(tf.transpose(concept),tf.matmul(self.W_b,self.memory))),axis=0)
        g_1 =  tf.nn.tanh(tf.add(tf.matmul(self.W_1,z_vector),self.b_1))
        g_scalar =  tf.nn.sigmoid(tf.add(tf.matmul(self.W_2,g_1),self.b_2))
        return g_scalar
    
            
        
        

In [35]:
tf.reset_default_graph()
g = tf.Graph()
with g.as_default():
    x = DMN_QA()
    x.build()

[<tf.Tensor 'add_939:0' shape=(100, 1) dtype=float64>, <tf.Tensor 'add_969:0' shape=(100, 1) dtype=float64>, <tf.Tensor 'add_999:0' shape=(100, 1) dtype=float64>, <tf.Tensor 'add_1029:0' shape=(100, 1) dtype=float64>]
None


In [ ]:
def QA2V(context_no,question_no):
    passage = TextBlob(df_contxt['context'][context_no])
    question = TextBlob(df_qas['Question'][question_no])
    if df_qas['is_impossible'][question_no]==False:
        answer = TextBlob(df_qas['Answer_text'][question_no])
        beginning = passage[:df_qas['Answer_start'][question_no]]
        end = passage[df_qas['Answer_start'][question_no]+len(answer):]  # verified
        
        
#         print(answer.words,'\n',len(answer.words),'\n',len(beginning.words),'\n',len(end.words))
    print(len(passage.words),'\n',len(beginning.words),'\n',len(answer.words),'\n',len(end.words))
        
    

In [ ]:
# QA2V(1,1)

In [2]:
# TextBlob(str(TextBlob(df_contxt['context'][1]).sentences[0])+" E-O-S").words

In [48]:
x = tf.constant(value=np.random.random(size=(100,1)),shape=(100,1))
y = tf.constant(value=np.random.random(size=(100,1)),shape=(100,1))
z = tf.tensordot(tf.transpose(x),y,axes=1)


In [49]:
z

<tf.Tensor 'Tensordot_3:0' shape=(1, 1) dtype=float64>